In [1]:
import numpy as np
import pandas as pd
from utils import dictionaries

In [2]:
filename="../data/Enterprise Statistics by Size Class Eurostat.xlsx"

In [3]:
sheets=pd.ExcelFile(path_or_buffer=filename).sheet_names
sheets

['Summary',
 'Structure',
 'Sheet 1',
 'Sheet 2',
 'Sheet 3',
 'Sheet 4',
 'Sheet 5',
 'Sheet 6',
 'Sheet 7',
 'Sheet 8',
 'Sheet 9',
 'Sheet 10',
 'Sheet 11',
 'Sheet 12',
 'Sheet 13',
 'Sheet 14',
 'Sheet 15',
 'Sheet 16',
 'Sheet 17',
 'Sheet 18',
 'Sheet 19',
 'Sheet 20',
 'Sheet 21',
 'Sheet 22',
 'Sheet 23',
 'Sheet 24']

In [4]:
def sheet_finder(sheet:str,filepath:str,n=8)->None:
    df=pd.read_excel(
        io=filepath,
        sheet_name=sheet,
        nrows=n
    )
    try:
        if ("Sheet" in sheet) and ("Enterprises - number" in df["Unnamed: 2"].values) and ("Total" not in df["Unnamed: 2"].values):
            return True
        else:
            return False
    except:
        return False

In [5]:
def sheet_reader(sheet:str,filepath:str)->None:
    df=pd.read_excel(
        io=filepath,
        sheet_name=sheet
    )
    segment=df.iloc[6]["Unnamed: 2"]
    columns=df.iloc[8].dropna()
    columns=["country"]+list(columns[columns!="TIME"].values)
    
    df=df.iloc[11:46].dropna(axis=1)
    df.columns=columns
    df["segment"]=segment
    df=df.apply(lambda col: col.replace(":",np.nan),axis=1)
    return df

In [6]:
sheet_flags=[]
for sheet in sheets:
    sheet_flags.append(sheet_finder(sheet=sheet,filepath=filename))

In [7]:
dfs=[]
for sheet,annotation in zip(sheets,sheet_flags):
    if annotation:
        dfs.append(sheet_reader(sheet=sheet,filepath=filename))

In [9]:
eurostat_data=pd.concat(dfs,axis=0)

In [12]:
eurostat_data["s_level_segment"]=eurostat_data["segment"].apply(
    lambda segment: dictionaries["eurostat_segments"].get(segment)
)

In [13]:
eurostat_data

,country,2021,2022,segment,s_level_segment
11,Belgium,NaN,NaN,From 0 to 1 person employed,S1-S3
12,Bulgaria,NaN,NaN,From 0 to 1 person employed,S1-S3
13,Czechia,NaN,NaN,From 0 to 1 person employed,S1-S3
14,Denmark,NaN,NaN,From 0 to 1 person employed,S1-S3
15,Germany,NaN,NaN,From 0 to 1 person employed,S1-S3
...,...,...,...,...,...
41,Bosnia and Herzegovina,277.0,290.0,250 persons employed or more,S7+
42,Montenegro,NaN,NaN,250 persons employed or more,S7+
43,North Macedonia,152.0,164.0,250 persons employed or more,S7+
44,Albania,NaN,NaN,250 persons employed or more,S7+
